In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Download a WFS as gml

The wfs downloaded is the muncipality of Amsterdam, Netherlands. The wfs is provided by [pdok](https://www.pdok.nl/introductie/-/article/bestuurlijke-grenzen).

In [2]:
import ogr
from owslib.etree import etree
from owslib.fes import PropertyIsLike
from owslib.wfs import WebFeatureService

from open_data import helpers

In [3]:
wfs_url = 'https://geodata.nationaalgeoregister.nl/bestuurlijkegrenzen/wfs'

In [4]:
wfs = WebFeatureService(wfs_url, version='1.1.0')
wfs.identification.title

'Bestuurlijke grenzen WFS'

In [5]:
print('The WFS has the following layers')
list(wfs.contents)

The WFS has the following layers


['bestuurlijkegrenzen:gemeenten',
 'bestuurlijkegrenzen:landsgrens',
 'bestuurlijkegrenzen:provincies']

We need the `'bestuurlijkegrenzen:gemeenten'` ('gemeenten' is 'municipalities' in Dutch).

In [6]:
layer = 'bestuurlijkegrenzen:gemeenten'
wfs.get_schema(layer)

{'properties': {'code': 'string', 'gemeentenaam': 'string'},
 'geometry': 'GeometryCollection',
 'geometry_column': 'geom'}

We will use the `'gemeentenaam'` property to filter for Amsterdam.

In [ ]:
muni = 'Amsterdam'
filter_ = PropertyIsLike(
    propertyname='gemeentenaam', 
    literal=muni, 
    matchCase=True,
)
filterxml = etree.tostring(filter_.toXML()).decode("utf-8")

response = wfs.getfeature(typename=layer, filter=filterxml)
file_path = f'../data/{muni}.gml'.lower()
with open(file_path, 'wb') as f:
    f.write(bytes(response.read(), 'UTF-8'))

Get the bounding box of the layer.

In [ ]:
driver = ogr.GetDriverByName('GML')
gml = driver.Open(file_path, 0)
layer = gml.GetLayer()
bbox = helpers.get_layer_extent(layer)
bbox